In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time

In [5]:
#Loading the webpage
url = 'https://stats.espncricinfo.com/ci/engine/stats/index.html?class=1;filter=advanced;orderby=runs;template=results;type=batting'
header = {'User Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
r = requests.get(url) #fetching the url
soup = BeautifulSoup(r.content,'lxml') #getting the content
print(soup.prettify()) #checking the data
Player_table = soup.find_all("table",class_ = 'engineTable')[2] #getting table from page

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<!-- hostname: web04, edition-view: , country: unknown, cluster: www, created: 2022-05-06 14:16:43 -->
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <script type="text/javascript">
   var _sf_startpt=(new Date()).getTime()
  </script>
  <meta content="ZxdgH3XglRg0Bsy-Ho2RnO3EE4nRs53FloLS6fkt_nc" name="google-site-verification"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
  <title>
   Batting records | Test matches | Cricinfo Statsguru | ESPNcricinfo.com
  </title>
  <meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
  <meta content="" name="keywords"/>
  <meta content="" name="description"/>
  <!--[if IE 9]>
<script language="javascript" type="text/javascript">
function fnCreateJumpLi

In [6]:
#Creating a template to extract data from the table in webpage

for name in Player_table.find_all('tbody'):
    rows = name.find_all('tr')
    for row in rows:
        player = row.find('td',class_ = 'left').text
        span = row.find_all('td')[1].text
        mat = row.find_all('td')[2].text
        inns = row.find_all('td')[3].text
        NO = row.find_all('td')[4].text
        runs = row.find_all('td')[5].text
        HS = row.find_all('td')[6].text
        Ave = row.find_all('td')[7].text
        Hundreds = row.find_all('td')[8].text
        Fifties = row.find_all('td')[9].text
        Zero = row.find_all('td')[10].text
        
        #Storing it in a dictionary
        Cricketers = {'Player':player,"Span":span,"Matches":mat,"Innings":inns,"NO":NO,"Runs":runs,"HS":HS,
                     "AVG":Ave,"100":Hundreds,"50":Fifties,"0":Zero}
        #print(Cricketers)


In [7]:
#Assigning an empty list to store all dictionaries of players

Cricketers_List = []

#Automating scrapping through different pages[the code has only first 2 page loop] by using a for loop and an f string edit in the link.

for page in range(1,63):
    source = requests.get(f'https://stats.espncricinfo.com/ci/engine/stats/index.html?class=1;filter=advanced;orderby=runs;page={page};template=results;type=batting')
    soup = BeautifulSoup(source.content,'lxml')
    Player_table = soup.find_all("table",class_ = 'engineTable')[2]
    
    #Code from the above cell which builds table structure
    
    for name in Player_table.find_all('tbody'):
        rows = name.find_all('tr')
        for row in rows:
            player = row.find('td',class_ = 'left').text
            span = row.find_all('td')[1].text
            mat = row.find_all('td')[2].text
            inns = row.find_all('td')[3].text
            NO = row.find_all('td')[4].text
            runs = row.find_all('td')[5].text
            HS = row.find_all('td')[6].text
            Ave = row.find_all('td')[7].text
            Hundreds = row.find_all('td')[8].text
            Fifties = row.find_all('td')[9].text
            Zero = row.find_all('td')[10].text
        
        #Storing it in a dictionary
            Cricketers = {'Player':player,"Span":span,"Matches":mat,"Innings":inns,"NO":NO,"Runs":runs,"HS":HS,
                     "AVG":Ave,"100":Hundreds,"50":Fifties,"0":Zero}
            Cricketers_List.append(Cricketers)
    
           # print('Saving: ', Cricketers['Player'])

            #time.sleep(3)

In [8]:
#Converting to DataFrame using pandas [sorted by number of runs by default]

df_testbatting = pd.DataFrame(Cricketers_List)
df_testbatting

,Player,Span,Matches,Innings,NO,Runs,HS,AVG,100,50,0
0,SR Tendulkar (INDIA),1989-2013,200,329,33,15921,248*,53.78,51,68,14
1,RT Ponting (AUS),1995-2012,168,287,29,13378,257,51.85,41,62,17
2,JH Kallis (ICC/SA),1995-2013,166,280,40,13289,224,55.37,45,58,16
3,R Dravid (ICC/INDIA),1996-2012,164,286,32,13288,270,52.31,36,63,8
4,AN Cook (ENG),2006-2018,161,291,16,12472,294,45.35,33,57,9
...,...,...,...,...,...,...,...,...,...,...,...
3076,Tabish Khan (PAK),2021-2021,1,-,-,-,-,-,-,-,-
3077,Usman Shinwari (PAK),2019-2019,1,-,-,-,-,-,-,-,-
3078,CM Willoughby (SA),2003-2003,2,-,-,-,-,-,-,-,-
3079,JW Wilson (AUS),1956-1956,1,-,-,-,-,-,-,-,-


In [12]:
df_testbatting.set_axis(['Player','TEST_Span','TEST_Matches','TEST_Innings','TEST_Not-Outs','TEST_RunsMade','TESTHighest-Score','TEST_Batting_Average','TEST_Hundereds','TEST_Fifties','TEST_Ducks'],axis=1,inplace=True)
df_testbatting=df_testbatting.replace('-', 0)
df_testbatting.to_csv("testbatting.csv")
df_testbatting


,Player,TEST_Span,TEST_Matches,TEST_Innings,TEST_Not-Outs,TEST_RunsMade,TESTHighest-Score,TEST_Batting_Average,TEST_Hundereds,TEST_Fifties,TEST_Ducks
0,SR Tendulkar (INDIA),1989-2013,200,329,33,15921,248*,53.78,51,68,14
1,RT Ponting (AUS),1995-2012,168,287,29,13378,257,51.85,41,62,17
2,JH Kallis (ICC/SA),1995-2013,166,280,40,13289,224,55.37,45,58,16
3,R Dravid (ICC/INDIA),1996-2012,164,286,32,13288,270,52.31,36,63,8
4,AN Cook (ENG),2006-2018,161,291,16,12472,294,45.35,33,57,9
...,...,...,...,...,...,...,...,...,...,...,...
3076,Tabish Khan (PAK),2021-2021,1,0,0,0,0,0,0,0,0
3077,Usman Shinwari (PAK),2019-2019,1,0,0,0,0,0,0,0,0
3078,CM Willoughby (SA),2003-2003,2,0,0,0,0,0,0,0,0
3079,JW Wilson (AUS),1956-1956,1,0,0,0,0,0,0,0,0


In [15]:
#'Player','Test_Span','Test_Matches','Test_Innings','Test_Not-Outs','Test_Runs',
#'TestHighest-Score',
#'Test_Avg','Test_Hundereds','Test_Fifties','Test_Ducks'

df_testbatting['TEST_Matches']=df_testbatting['TEST_Matches'].astype(int)
df_testbatting['TEST_Innings']=df_testbatting['TEST_Innings'].astype(int)
df_testbatting['TEST_Not-Outs']=df_testbatting['TEST_Not-Outs'].astype(int)
df_testbatting['TEST_RunsMade']=df_testbatting['TEST_RunsMade'].astype(int)
df_testbatting['TEST_Hundereds']=df_testbatting['TEST_Hundereds'].astype(int)
df_testbatting['TEST_Fifties']=df_testbatting['TEST_Fifties'].astype(int)
df_testbatting['TEST_Ducks']=df_testbatting['TEST_Ducks'].astype(int)
df_testbatting["TEST_Batting_Average"] = pd.to_numeric(df_testbatting['TEST_Batting_Average'], errors='coerce')
df_testbatting.dtypes


Player                   object
TEST_Span                object
TEST_Matches              int32
TEST_Innings              int32
TEST_Not-Outs             int32
TEST_RunsMade             int32
TESTHighest-Score        object
TEST_Batting_Average    float64
TEST_Hundereds            int32
TEST_Fifties              int32
TEST_Ducks                int32
dtype: object

In [16]:
df_testbatting

,Player,TEST_Span,TEST_Matches,TEST_Innings,TEST_Not-Outs,TEST_RunsMade,TESTHighest-Score,TEST_Batting_Average,TEST_Hundereds,TEST_Fifties,TEST_Ducks
0,SR Tendulkar (INDIA),1989-2013,200,329,33,15921,248*,53.78,51,68,14
1,RT Ponting (AUS),1995-2012,168,287,29,13378,257,51.85,41,62,17
2,JH Kallis (ICC/SA),1995-2013,166,280,40,13289,224,55.37,45,58,16
3,R Dravid (ICC/INDIA),1996-2012,164,286,32,13288,270,52.31,36,63,8
4,AN Cook (ENG),2006-2018,161,291,16,12472,294,45.35,33,57,9
...,...,...,...,...,...,...,...,...,...,...,...
3076,Tabish Khan (PAK),2021-2021,1,0,0,0,0,0.00,0,0,0
3077,Usman Shinwari (PAK),2019-2019,1,0,0,0,0,0.00,0,0,0
3078,CM Willoughby (SA),2003-2003,2,0,0,0,0,0.00,0,0,0
3079,JW Wilson (AUS),1956-1956,1,0,0,0,0,0.00,0,0,0


# 

In [17]:
df_testbatting.to_excel('test Batsmenv1.xlsx')

In [11]:
#exporting the file as not running the complete code again
df_testbatting2= pd.read_excel('test Batsmenv1.xlsx')

In [12]:
df_testbatting2

,Unnamed: 0,Player,TEST_Span,TEST_Matches,TEST_Innings,TEST_Not-Outs,TEST_RunsMade,TESTHighest-Score,TEST_Batting_Average,TEST_Hundereds,TEST_Fifties,TEST_Ducks
0,0,SR Tendulkar (INDIA),1989-2013,200,329,33,15921,248*,53.78,51,68,14
1,1,RT Ponting (AUS),1995-2012,168,287,29,13378,257,51.85,41,62,17
2,2,JH Kallis (ICC/SA),1995-2013,166,280,40,13289,224,55.37,45,58,16
3,3,R Dravid (ICC/INDIA),1996-2012,164,286,32,13288,270,52.31,36,63,8
4,4,AN Cook (ENG),2006-2018,161,291,16,12472,294,45.35,33,57,9
...,...,...,...,...,...,...,...,...,...,...,...,...
3076,3076,Tabish Khan (PAK),2021-2021,1,0,0,0,0,0.00,0,0,0
3077,3077,Usman Shinwari (PAK),2019-2019,1,0,0,0,0,0.00,0,0,0
3078,3078,CM Willoughby (SA),2003-2003,2,0,0,0,0,0.00,0,0,0
3079,3079,JW Wilson (AUS),1956-1956,1,0,0,0,0,0.00,0,0,0


In [13]:
#dropping the unnamed index coming
df_testbatting2=df_testbatting2.drop(['Unnamed: 0'],axis=1)
df_testbatting2

,Player,TEST_Span,TEST_Matches,TEST_Innings,TEST_Not-Outs,TEST_RunsMade,TESTHighest-Score,TEST_Batting_Average,TEST_Hundereds,TEST_Fifties,TEST_Ducks
0,SR Tendulkar (INDIA),1989-2013,200,329,33,15921,248*,53.78,51,68,14
1,RT Ponting (AUS),1995-2012,168,287,29,13378,257,51.85,41,62,17
2,JH Kallis (ICC/SA),1995-2013,166,280,40,13289,224,55.37,45,58,16
3,R Dravid (ICC/INDIA),1996-2012,164,286,32,13288,270,52.31,36,63,8
4,AN Cook (ENG),2006-2018,161,291,16,12472,294,45.35,33,57,9
...,...,...,...,...,...,...,...,...,...,...,...
3076,Tabish Khan (PAK),2021-2021,1,0,0,0,0,0.00,0,0,0
3077,Usman Shinwari (PAK),2019-2019,1,0,0,0,0,0.00,0,0,0
3078,CM Willoughby (SA),2003-2003,2,0,0,0,0,0.00,0,0,0
3079,JW Wilson (AUS),1956-1956,1,0,0,0,0,0.00,0,0,0


In [14]:
#As checked we are finding 2 major errors in player column.. 1st is some player are having name+(numericvalue)+(countryname)
#2nd having ICC/World/Asia/Afr along with country name
#1st removing (numeric data)
df_testbatting2['Player']=df_testbatting2['Player'].str.replace(r"\(.1?\)","")
df_testbatting2['Player']=df_testbatting2['Player'].str.replace(r"\(.2?\)","")
df_testbatting2['Player']=df_testbatting2['Player'].str.replace(r"\(.3?\)","")

<ipython-input-14-480c3596e1a7>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_testbatting2['Player']=df_testbatting2['Player'].str.replace(r"\(.1?\)","")
<ipython-input-14-480c3596e1a7>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_testbatting2['Player']=df_testbatting2['Player'].str.replace(r"\(.2?\)","")
<ipython-input-14-480c3596e1a7>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_testbatting2['Player']=df_testbatting2['Player'].str.replace(r"\(.3?\)","")


In [15]:
#2nd replacing ICC/World/Asia/Afr from the country name

df_testbatting2= df_testbatting2.replace('ICC/', '', regex=True)
df_testbatting2= df_testbatting2.replace('/ICC', '', regex=True)
df_testbatting2= df_testbatting2.replace('/Afr', '', regex=True)
df_testbatting2= df_testbatting2.replace('Afr/', '', regex=True)
df_testbatting2= df_testbatting2.replace('/Asia', '', regex=True)  
df_testbatting2= df_testbatting2.replace('Asia/', '', regex=True)  
df_testbatting2= df_testbatting2.replace('/World', '', regex=True)    


In [16]:
df_testbatting2

,Player,TEST_Span,TEST_Matches,TEST_Innings,TEST_Not-Outs,TEST_RunsMade,TESTHighest-Score,TEST_Batting_Average,TEST_Hundereds,TEST_Fifties,TEST_Ducks
0,SR Tendulkar (INDIA),1989-2013,200,329,33,15921,248*,53.78,51,68,14
1,RT Ponting (AUS),1995-2012,168,287,29,13378,257,51.85,41,62,17
2,JH Kallis (SA),1995-2013,166,280,40,13289,224,55.37,45,58,16
3,R Dravid (INDIA),1996-2012,164,286,32,13288,270,52.31,36,63,8
4,AN Cook (ENG),2006-2018,161,291,16,12472,294,45.35,33,57,9
...,...,...,...,...,...,...,...,...,...,...,...
3076,Tabish Khan (PAK),2021-2021,1,0,0,0,0,0.00,0,0,0
3077,Usman Shinwari (PAK),2019-2019,1,0,0,0,0,0.00,0,0,0
3078,CM Willoughby (SA),2003-2003,2,0,0,0,0,0.00,0,0,0
3079,JW Wilson (AUS),1956-1956,1,0,0,0,0,0.00,0,0,0


In [17]:
df_testbatting2.to_excel('Test battingv3.xlsx')